# MONAIのモデル採用した基本的なセグメンテーション

本チュートリアルではniiデータを読み込み、MONAIの2D-UNETモデルを利用して、スライス単位でのセグメンテーション学習を学びます。
ここでは、Pytorchを利用した基本的な学習方法や推論方法について覚えてください。

## 必要なライブラリのインストールを行います

In [ ]:
!pip install -U SimpleITK "monai[ignote, nibabel, torchvision, tdqm]==1.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00


## 学習用サンプルデータおよび実装表示用関数等のコードをダウンロードして解凍しましょう

In [ ]:
# niiサンプルデータをダウンロード(Lungのセグメンテーション学習用データ約10症例)
# このデータはTCIAのパブリックデータを加工して提供しています
!wget --no-clobber https://www.dropbox.com/s/f6jk58w6tbca31r/nii.zip

--2023-09-05 09:29:07--  https://www.dropbox.com/s/f6jk58w6tbca31r/nii.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/f6jk58w6tbca31r/nii.zip [following]
--2023-09-05 09:29:07--  https://www.dropbox.com/s/raw/f6jk58w6tbca31r/nii.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc079a0f9939dc68f7950859d53c.dl.dropboxusercontent.com/cd/0/inline/CDKJMaIb1BZJQN1yfvf5WfYKYc4lFbj4Qavwvtv2bMHbs_qnybP9yNHBVtEkIoJMeXxdZQF-Il9Wwu_cT5WTfnMrahUNKDiF2X9QWFoQo3Ho_brtVGjROXNO0ryQm6vha-c/file# [following]
--2023-09-05 09:29:08--  https://uc079a0f9939dc68f7950859d53c.dl.dropboxusercontent.com/cd/0/inline/CDKJMaIb1BZJQN1yfvf5WfYKYc4lFbj4Qavwvtv2bMHbs_qnybP9yNHBVtEkIoJMeXxdZQF-Il9Wwu_cT5WTfnMrahUNKDiF2X9QWFoQo3Ho_brtVGjROXNO0ryQm6vha-c/file
R

In [ ]:
# ダウンロードしたサンプルファイルを解凍します
!unzip -n nii.zip

Archive:  nii.zip
  inflating: nii/0000_ct.nii.gz      
  inflating: nii/0000_Lung.nii.gz    
  inflating: nii/0001_ct.nii.gz      
  inflating: nii/0001_Lung.nii.gz    
  inflating: nii/0002_ct.nii.gz      
  inflating: nii/0002_Lung.nii.gz    
  inflating: nii/0003_ct.nii.gz      
  inflating: nii/0003_Lung.nii.gz    
  inflating: nii/0004_ct.nii.gz      
  inflating: nii/0004_Lung.nii.gz    
  inflating: nii/0005_ct.nii.gz      
  inflating: nii/0005_Lung.nii.gz    
  inflating: nii/0006_ct.nii.gz      
  inflating: nii/0006_Lung.nii.gz    
  inflating: nii/0007_ct.nii.gz      
  inflating: nii/0007_Lung.nii.gz    
  inflating: nii/0008_ct.nii.gz      
  inflating: nii/0008_Lung.nii.gz    
  inflating: nii/0009_ct.nii.gz      
  inflating: nii/0009_Lung.nii.gz    


In [ ]:
# 表示用等で実装済み関数を含むPythonファイルをダウンロードして展開します
!rm -r seminor_inc
!rm seminor_inc.zip
!wget -O seminor_inc.zip --no-clobber https://www.dropbox.com/scl/fi/e9ywrbheofjw2k8jxvtmp/seminor_inc.zip?rlkey=5ri24j9n3tc89llvabjg2g2i9&dl=0
!unzip -n seminor_inc.zip

--2023-09-05 09:49:02--  https://www.dropbox.com/scl/fi/e9ywrbheofjw2k8jxvtmp/seminor_inc.zip?rlkey=5ri24j9n3tc89llvabjg2g2i9
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/e9ywrbheofjw2k8jxvtmp/seminor_inc.zip?rlkey=5ri24j9n3tc89llvabjg2g2i9 [following]
--2023-09-05 09:49:02--  https://www.dropbox.com/e/scl/fi/e9ywrbheofjw2k8jxvtmp/seminor_inc.zip?rlkey=5ri24j9n3tc89llvabjg2g2i9
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5953e4e6a6ad96238797e08526.dl.dropboxusercontent.com/cd/0/inline/CDKE0F7v_D55MNneOiVA_9aAIAFifPLR473DUv0Jk-bH42b4JjRwB91vgCWwT6egW28iYw3r0UAIZoZxYn54MdNIqrUtE04e2_jEuWGoyojCFD51wUXKbgSZsuJg58CUiZw/file# [following]
--2023-09-05 09:49:03--  https://uc5953e4e6a6ad96238797e08526.dl.d

## データ読み込み
niiファイルをSimpleITKで読み込み、学習用と検証用に8:2で分割しておきます。

In [ ]:


import os, sys, glob
import numpy as np
import cv2
import SimpleITK as sitk
sys.path.append("seminor_inc/")     #seminor_incフォルダ以下のソースコードも参照できるように設定
from my_functions import *
from unet_torch import *


####### データの前処理 #######

dataDir = "nii/"  #データフォルダ

#CTファイル一覧を取得
ctRawList = glob.glob(dataDir + "*ct.nii.gz")
ctRawList.sort()

### データ読み込み ###
dataX = []
dataY = []
for rawPath in ctRawList:
    # 画像を読み込む
    image = sitk.ReadImage(rawPath)
    # SimpleITK Imageをnumpy配列に変換する
    rawCT = sitk.GetArrayFromImage(image)
    print(rawPath, rawCT.shape)

    rawLungPath = rawPath.replace("ct.nii.gz", "Lung.nii.gz")
    # ラベル画像を読み込む
    image = sitk.ReadImage(rawLungPath)
    # SimpleITK Imageをnumpy配列に変換する
    rawLung = sitk.GetArrayFromImage(image)

    orgShape = rawCT.shape

    #学習用画像及び教師データは(Z, Y, X)の形で保存されていて
    #ともにグレースケールの1チャンネルであるので、チャンネル数情報として追加しておく
    shape = (1, orgShape[0], orgShape[1], orgShape[2])

    #4次元配列(Channel, Z, Y, X)にリサイズ
    rawCT = rawCT.reshape(tuple(shape))
    rawLung = rawLung.reshape(tuple(shape))

    rawCT = np.clip(rawCT, -1000, 1000)     #CT値は範囲が広すぎるので、対象臓器が含まれる範囲にクリップ

    ### 学習用にCT値を 0から1の範囲に正規化
    ### これは好きな範囲で良い(例えば-1から1の範囲に正規化しても良い)
    rawCTnorm = (rawCT.astype("float") + 1000.0) / 2000.0

    ### 教師用データを0～1の範囲に正規化
    rawLungNorm = rawLung.astype("float")

    dataX.append(rawCTnorm)     #CTを学習時の入力用配列に登録
    dataY.append(rawLungNorm)   #CTを学習時の教師用配列に登録

    if False:
        ### 試しに中央のスライスとそのLung領域を表示してみる
        slicePos = int(rawCT.shape[1] / 2)          #中間の位置を算出
        sliceCT = rawCT[0, slicePos, :, :]             #CTスライスを取り出す
        sliceUint8 = getSliceByLut(sliceCT, wl=0, ww=2000)  #WLとWWを指定して表示用画像に変換
        sliceLung = rawLung[0, slicePos, :, :]         #同じ位置のLung領域を取り出す

        img = np.hstack((sliceUint8.reshape((256, 256)), sliceLung.reshape((256, 256)))).astype("uint8")
        import matplotlib.pyplot as plt
        plt.imshow(img)
        plt.gray()
        plt.show()



### 学習用と検証用にデータを分割 ###
trainNum = int( len(dataX) * 0.8 )  #学習用データ数。今回は8割(10データ中8データ)を学習に使う
testNum = len(dataX) - trainNum     #検証用データ数

trainX = dataX[:trainNum]  # 学習用入力
trainY = dataY[:trainNum]  # 学習用教師データ
testX = dataX[trainNum:]  # テスト用入力
testY = dataY[trainNum:]  # テスト用教師データ


#本当に正規化が正しく行われているかを最初のデータで見てみる
print("dataX[0]", np.min(trainX[0]), np.max(trainX[0]))
print("dataY[0]", np.min(trainY[0]), np.max(trainY[0]))



nii/0000_ct.nii.gz (277, 256, 256)
nii/0001_ct.nii.gz (261, 256, 256)
nii/0002_ct.nii.gz (241, 256, 256)
nii/0003_ct.nii.gz (256, 256, 256)
nii/0004_ct.nii.gz (133, 256, 256)
nii/0005_ct.nii.gz (133, 256, 256)
nii/0006_ct.nii.gz (133, 256, 256)
nii/0007_ct.nii.gz (133, 256, 256)
nii/0008_ct.nii.gz (140, 256, 256)
nii/0009_ct.nii.gz (145, 256, 256)
dataX[0] 0.0 1.0
dataY[0] 0.0 1.0


## 学習実施

セグメンテーション学習では領域一致度の指標として、DICE係数を用います。学習サイクルではDice係数の最大化(つまり1.0-Dice係数の最小化)を目指します。

In [ ]:

####### 損失関数およびモデルの定義 #######
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import monai

# ダイス係数を計算する関数
def dice_coefficient(pred, target):
    smooth = 1e-6
    num = pred.size(0)
    m1 = pred.view(num, -1)
    m2 = target.view(num, -1)
    intersection = (m1 * m2).sum()

    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)

# 損失関数
def dice_loss(pred, target):
    return 1. - dice_coefficient(pred, target)

#学習用デバイスを決定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#モデルを読み込み
model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)

#損失関数はダイス係数(1.0より小さいほど損失大)、オプティマイザーはAdamを利用
criterion = dice_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
####### 学習の実施 #######
# 学習したモデルの保存用フォルダを生成
modelDir = "model/"
os.makedirs(modelDir, exist_ok=True)


# 学習するエポック数とStep数
# モデルの重みの総修正回数は エポック数×ステップ数となります
totalEpoch = 10  #エポック数
steps = 200     #1エポック内の学習回数

# Stepあたりのバッチサイズ
# 学習ステップごとにモデルに供給するデータサイズであり、大きすぎるとメモリ不足で学習プロセスが止まります
batchSize = 9

#学習済み重みをリロードして、続きから学習する場合
wPath = "model/unet_weights.pth"
if False:
    model.load_state_dict(torch.load(wPath))

# totalEpoch - fromEpoch回のエポックを繰り返す
for iEpoch in range(totalEpoch):

    # [Pytorch学習の決まりごと1]モデルを学習モードに設定
    model.train()

    # エポックごとにsteps回の学習を実施
    # つまり、steps x fromEpoch 回の学習と重み修正が行われる
    for iStep in range(steps):

        # [Pytorch学習の決まりごと2]勾配を0にリセット
        optimizer.zero_grad()

        trainBatchX, trainBatchY = [], []
        dataIds = np.random.randint(0, trainNum, 3)         #ステップごとに3つデータをランダムに決定
        ### データごとにスライスを任意に取得する処理
        for dataId in dataIds:
            sliceNum = trainX[dataId].shape[1]              #そのデータ内のスライス数を参照
            sliceIds = np.random.randint(0, sliceNum, 3)    #当該データ内で3スライスをランダムに決定
            for sliceId in sliceIds:
                trainBatchX.append( trainX[dataId][:, sliceId] )       #バッチに正規化済みスライスCTを入力として加える
                trainBatchY.append( trainY[dataId][:, sliceId] )       #バッチに正規化済みLung領域を教師データとして加える

        trainBatchX = np.array(trainBatchX)
        trainBatchY = np.array(trainBatchY)
        if False:
            ### 試しに中央のスライスとそのLung領域を表示してみる
            sliceCT = trainBatchX[0, 0, :, :] * 1000             #CTスライスを取り出す
            sliceUint8 = getSliceByLut(sliceCT, wl=0, ww=2000)  #WLとWWを指定して表示用画像に変換
            sliceLung = trainBatchY[0, 0, :, :] *255        #同じ位置のLung領域を取り出す

            img = np.hstack((sliceUint8.reshape((256, 256)), sliceLung.reshape((256, 256)))).astype("uint8")
            import matplotlib.pyplot as plt
            plt.imshow(img)
            plt.gray()
            plt.show()

        # モデルに渡す際は必ずPyTorchのテンソルに変換
        trainBatchX = torch.tensor(trainBatchX, dtype=torch.float32).to(device)
        trainBatchY = torch.tensor(trainBatchY, dtype=torch.float32).to(device)
        #print("batch size",trainBatchX.shape, trainBatchY.shape)

        ### 取得したバッチで学習実施

        # [Pytorch学習の決まりごと3]モデルに入力を渡して出力を得る
        outputs = model(trainBatchX)

        # [Pytorch学習の決まりごと4]モデルの出力に対して損失を計算
        loss = criterion(torch.sigmoid(outputs), trainBatchY)

        # [Pytorch学習の決まりごと5]損失から逆伝播して損失の勾配を計算
        loss.backward()

        # [Pytorch学習の決まりごと6]計算された損失勾配からモデルを更新
        optimizer.step()

        if (iStep + 1) % 50 == 0:
          print(f"Epoch {iEpoch + 1}/{totalEpoch}, Step {iStep+1}/{steps}, Loss: {loss.item()}")


    #学習した重みを保存
    torch.save(model.state_dict(), wPath)

    #エポックごとに学習が終わったら, テスト用データで検証してみる

    # [Pytorch検証の決まりごと1]モデルを推論モードに設定
    model.eval()
    total_loss = 0
    ### testデータごとにスライスを任意に取得する処理
    for dataId in range(testNum):
        testBatchX, testBatchY = [], []
        sliceNum = testX[dataId].shape[1]  # そのデータ内のスライス数を参照
        #sliceIds = np.random.randint(0, sliceNum, )  # 当該データ内で3スライスをランダムに決定
        for sliceId in range(sliceNum):
            testBatchX.append(testX[dataId][:, sliceId])  # バッチに正規化済みスライスCTを入力として加える
            testBatchY.append(testY[dataId][:, sliceId])  # バッチに正規化済みLung領域を評価用教師データとして加える

        # モデルに渡す際は必ずPyTorchのテンソルに変換
        testBatchX = np.array(testBatchX)
        testBatchY = np.array(testBatchY)
        testBatchX = torch.tensor(testBatchX, dtype=torch.float32).to(device)
        testBatchY = torch.tensor(testBatchY, dtype=torch.float32).to(device)

        # 評価実施
        outputs_test = model(testBatchX)
        loss_test = criterion(torch.sigmoid(outputs_test), testBatchY)
        total_loss += loss_test.item()
        print(f"testData:{dataId} shape:{testBatchX.shape} Loss:{loss_test.item()}")

    avg_loss = total_loss / testNum
    print(f"Epoch {iEpoch + 1}/{totalEpoch}, Test Loss: {avg_loss}\n")


Epoch 1/10, Step 50/200, Loss: 0.7032331228256226
Epoch 1/10, Step 100/200, Loss: 0.5510829091072083
Epoch 1/10, Step 150/200, Loss: 0.4847935438156128
Epoch 1/10, Step 200/200, Loss: 0.5178120136260986
testData:0 shape:torch.Size([140, 1, 256, 256]) Loss:0.615882158279419
testData:1 shape:torch.Size([145, 1, 256, 256]) Loss:0.6227025985717773
Epoch 1/10, Test Loss: 0.6192923784255981

Epoch 2/10, Step 50/200, Loss: 0.4605448842048645
Epoch 2/10, Step 100/200, Loss: 0.5527752637863159
Epoch 2/10, Step 150/200, Loss: 0.43612730503082275
Epoch 2/10, Step 200/200, Loss: 0.6471134424209595
testData:0 shape:torch.Size([140, 1, 256, 256]) Loss:0.4355754256248474
testData:1 shape:torch.Size([145, 1, 256, 256]) Loss:0.3670748472213745
Epoch 2/10, Test Loss: 0.40132513642311096

Epoch 3/10, Step 50/200, Loss: 0.6978133916854858
Epoch 3/10, Step 100/200, Loss: 0.24665534496307373
Epoch 3/10, Step 150/200, Loss: 0.3396301865577698
Epoch 3/10, Step 200/200, Loss: 0.19579726457595825
testData:0 sha

## 学習済みモデルの検証と推論結果の表示

In [ ]:
from numpy.core import einsumfunc

# 検証用データに対する予測結果
import monai
import matplotlib.pyplot as plt

def np_dice(pred, target, smooth=1e-08):
    pred = pred.flatten()
    target = target.flatten()

    intersection = np.sum(pred * target)

    dice = ((2.0 * intersection + smooth) / (np.sum(pred) + np.sum(target) + smooth))

    return dice

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)
#指定した重みを読み込み
wPath = "model/unet_weights.pth"
model.load_state_dict(torch.load(wPath))
model.eval()

diceList = []
for dataId in range(testNum):
    testBatchX, testBatchY = [], []
    sliceNum = testX[dataId].shape[1]  # そのデータ内のスライス数を参照
    #sliceIds = np.random.randint(0, sliceNum, )  # 当該データ内で3スライスをランダムに決定
    for sliceId in range(sliceNum):
        testBatchX.append(testX[dataId][:, sliceId])  # バッチに正規化済みスライスCTを入力として加える
        testBatchY.append(testY[dataId][:, sliceId])  # バッチに正規化済みLung領域を評価用教師データとして加える

    # モデルに渡す際は必ずPyTorchのテンソルに変換
    testBatchX = np.array(testBatchX)
    testBatchY = np.array(testBatchY)
    testBatchX = torch.tensor(testBatchX, dtype=torch.float32).to(device)
    testBatchY = torch.tensor(testBatchY, dtype=torch.float32).to(device)

    # 評価実施
    outputs_test = model(testBatchX)

    # Pytorchのテンソルからnumpy配列に変換する
    testBatchX = testBatchX.detach().cpu().numpy()
    testBatchY = testBatchY.detach().cpu().numpy()
    outputs_test = torch.sigmoid(outputs_test).detach().cpu().numpy()

    # 推論結果に対して、0.5をしきい値として後処理する
    outputs_test[outputs_test >= 0.5] = 1.0
    outputs_test[outputs_test < 0.5] = 0

    # 正解ラベルと推論結果のDICE係数を計算
    diceList.append(np_dice(outputs_test, testBatchY))

    if True:
      # 実装済み表示用関数で推論結果を表示する
      spacingX, spacingY, spacingZ = 1.0, 1.0, 1.0
      browse_result(testBatchX[:, 0], testBatchY[:, 0], outputs_test[:, 0], aspect=spacingY/spacingX)

#すべてのDICE
for i, dice in enumerate(diceList):
  print(f"test data {i} : {dice:.4f}")
print(f"test data mean dice : {np.mean(diceList):.4f}")


interactive(children=(IntSlider(value=69, description='i', max=139), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=72, description='i', max=144), Output()), _dom_classes=('widget-interact…

test data 0 : 0.9911
test data 1 : 0.9766
test data mean dice : 0.9839
